# OSS-DBS v2.0 Tutorial
At first import all needed modules. Besides OSS-DBS we  import the Draw function from Netgen/NGSolve webgui to visualze the results within the jupyter-notebook. The Taskmanager is used to paralize the computational expensive steps. If you want to use the default values for not-specified variables, you can use the `Settings` function from the `ossdbs.utils` module. 

In [ ]:
import logging

from netgen.webgui import Draw
from ngsolve import TaskManager
from ngsolve.webgui import Draw as DrawNG

import ossdbs
from ossdbs.utils.settings import Settings

ossdbs.set_logger(logging.INFO)

## Input Settings
Define settings in dictionary format. Also an JSON file can be loaded.

In [ ]:
settings = {
    "BrainRegion": {
        "Center": {"x[mm]": 17.0, "y[mm]": 8.0, "z[mm]": 6.0},
        "Dimension": {"x[mm]": 50.0, "y[mm]": 50.0, "z[mm]": 50.0},
        "Shape": "Ellipsoid",
    },
    "Electrodes": [
        {
            "Name": "BostonScientificVerciseCustom",
            "CustomParameters": {
                "tip_length": 1.3,
                "contact_length": 1.5,
                "contact_spacing": 0.5,
                "lead_diameter": 1.3,
                "total_length": 50.0,
            },
            "Rotation[Degrees]": 0.0,
            "Direction": {"x[mm]": 0.0, "y[mm]": 0.0, "z[mm]": 1.0},
            "TipPosition": {"x[mm]": 17.0, "y[mm]": 8.0, "z[mm]": 6.0},
            "Contacts": [
                {
                    "Contact_ID": 1,
                    "Active": True,
                    "Current[A]": 0.0,
                    "Voltage[V]": 1.0,
                    "Floating": False,
                    "SurfaceImpedance[Ohmm]": {"real": 0.0, "imag": 0.0},
                    "MaxMeshSizeEdge": 0.01,
                },
                {
                    "Contact_ID": 2,
                    "Active": True,
                    "Current[A]": 0.0,
                    "Voltage[V]": 0.0,
                    "Floating": False,
                    "SurfaceImpedance[Ohmm]": {"real": 0.0, "imag": 0.0},
                    "MaxMeshSizeEdge": 0.01,
                },
            ],
            "EncapsulationLayer": {
                "Thickness[mm]": 0.0,
                "Material": "Blood",
                "DielectricModel": "ColeCole4",
                "MaxMeshSize": 0.5,
            },
        }
    ],
    "MaterialDistribution": {
        "MRIPath": "../input_files/Butenko_segmask.nii.gz",
        "MRIMapping": {
            "Unknown": 0,
            "CSF": 1,
            "White matter": 2,
            "Gray matter": 3,
            "Blood": 4,
        },
        "DiffusionTensorActive": False,
        "DTIPath": "",
    },
    "DielectricModel": {"Type": "ColeCole4", "CustomParameters": None},
    "Mesh": {
        "LoadMesh": False,
        "MeshElementOrder": 2,
        "MeshingHypothesis": {"Type": "Default", "MaxMeshSize": 10.0},
        "MeshSize": {"Edges": {}, "Faces": {"E1C1": 0.1}, "Volumes": {"Brain": 0.5}},
        "SaveMesh": False,
    },
    "FEMOrder": 2,
    "EQSMode": False,
    "StimulationSignal": {
        "CurrentControlled": False,
        "Type": "Multisine",
        "ListOfFrequencies": [130.0],
    },
    "Solver": {
        "Type": "CG",
        "Preconditioner": "bddc",
        "PreconditionerKwargs": {},
        "PrintRates": False,
        "MaximumSteps": 10000,
        "Precision": 1e-12,
    },
    "PointModel": {
        "Pathway": {"Active": False, "FileName": ""},
        "Lattice": {
            "Active": False,
            "Center": {"x[mm]": 22.95, "y[mm]": 11.47, "z[mm]": 8.1},
            "Shape": {"x": 10, "y": 10, "z": 10},
            "Direction": {"x[mm]": 0, "y[mm]": 0, "z[mm]": 1},
            "PointDistance[mm]": 0.1,
        },
        "VoxelLattice": {
            "Active": False,
            "Shape": {"x": 10, "y": 10, "z": 10},
        },
    },
    "OutputPath": "Results/",
    "ComputeImpedance": True,
    "TemplateSpace": False,
    "ActivationThresholdVTA": 0.2,
    "ExportVTK": True,
    "ExportElectrode": True,
    "OutOfCore": False,
    "PathwayFile": None,
    "AdaptiveMeshRefinement": False,
}

## Update Settings
The following command will update all variables, which haven't been specified by the user with the default values.

In [ ]:
settings = Settings(settings).complete_settings()

## Load MRI and DTI Data
MRI and DTI were load from the path given in the settings dictionary. You always need to provide a MRI image, but DTI is optional.

In [ ]:
mri_image, dti_image = ossdbs.load_images(settings)

## Create Stimulation Electrodes
The specified electrodes were create and drawed. Geometrical parameters of the electrode can be changed directly in the input settings by appending "Custom" to the end of the electrodes' name and providing a dictionary with custum parameters.

In [ ]:
electrodes = ossdbs.generate_electrodes(settings)
Draw(electrodes[0].geometry)

## Construct Geometry of the Brain
Since the region of interesst is relativly small compared to the brain size, the shape can be approximated by a ellipsoide, sphere, or a box. Also providing a custom shape in `.brep` format is possible.

In [ ]:
region_parameters = settings["BrainRegion"]
brain_region = ossdbs.create_bounding_box(settings["BrainRegion"])
shape = settings["BrainRegion"]["Shape"]

brain = ossdbs.BrainGeometry(shape, brain_region)

In [ ]:
geometry = ossdbs.ModelGeometry(brain, electrodes)
Draw(geometry.geometry.shape)

## Prepare the Volume Conductor Model
Before running the simulation, the material properties and corresponding conductivity values are determined based on the given inputs. Also the settings for the solver can be updated.

In [ ]:
ossdbs.set_contact_and_encapsulation_layer_properties(settings, geometry)

dielectric_model = ossdbs.prepare_dielectric_properties(settings)

materials = settings["MaterialDistribution"]["MRIMapping"]
conductivity = ossdbs.ConductivityCF(
    mri_image,
    brain_region,
    dielectric_model,
    materials,
    geometry.encapsulation_layers,
    complex_data=settings["EQSMode"],
)

solver = ossdbs.prepare_solver(settings)

frequency_domain_signal = ossdbs.prepare_stimulation_signal(settings)

## Run Volume Conductor Model
To run the Volume Conductor Model, the built-in TaskManager is use to parallize the computational heavy parts. During this step, the results will be stored in the specified folder.

In [ ]:
with TaskManager():
    volume_conductor = ossdbs.prepare_volume_conductor_model(
        settings, geometry, conductivity, solver
    )
    ossdbs.run_volume_conductor_model(
        settings, volume_conductor, frequency_domain_signal
    )

In [ ]:
DrawNG(volume_conductor.potential)